In [17]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from datetime import datetime
# Using plotly.express
import plotly.express as px
import pandas as pd

### layout = what it looks like

### layout = a tree of three components

#### 

children = the first attribute of html.<ITEM>()
    
hot reloading = changes will be implemented like in live server

In [18]:
#DOWNLOAD FROM GOVT WEBSITE: 
#first 5 rows are header, first two columns are metadata

filepath = "govt_data/mortgage_rate_jan-2019_feb-2020.csv"
mortgage_df = pd.read_csv(filepath, skiprows=5)
mortgage_df.head()



,Unnamed: 0,Unnamed: 1,Canada,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Components,Unit of measure,January 2019,February 2019,March 2019,April 2019,May 2019,June 2019,July 2019,August 2019,September 2019,October 2019,November 2019,December 2019,January 2020,February 2020
1,"Total, funds advanced, residential mortgages, ...","Dollars (x 1,000,000)","5,916","5,844","6,529","8,195","10,357","11,752","12,310","11,008","10,072","10,248","9,744","8,043","7,572","7,970"
2,NaN,Interest rate,3.84,3.82,3.76,3.59,3.45,3.33,3.22,3.16,3.12,3.12,3.13,3.23,3.24,3.19
3,"Variable rate, funds advanced, residential mor...","Dollars (x 1,000,000)",955,781,783,748,761,640,554,463,382,363,374,384,366,422
4,NaN,Interest rate,3.52,3.66,3.75,3.82,3.90,3.99,4.08,4.15,4.17,4.19,4.01,3.97,3.91,3.93


In [19]:
def parse_govt_data(df, daterow_ind, valuerow_ind, datetime_format_in):
    """returns a list of datetime strings and financial indicator values from 
    dataframe of govt data
    
    assume datetime format for output datetime list = YYYY-MM-DD
    
    """
    date_row = df.iloc[daterow_ind]
    date_row = date_row[2:]
    date_list = date_row.tolist()
    datetime_list = []

    for item in date_list:
        dt = datetime.strptime(item, datetime_format_in)
        datetime_list.append(dt.strftime("%Y-%m-%d"))


    value_row = df.iloc[valuerow_ind]
    value_row = value_row[2:]
    value_list = value_row.tolist()
    
    assert (len(value_list) == len(datetime_list))

    return datetime_list, value_list



#### Mortgage Rate

In [20]:
datetime_mortgage, values_mortgage = parse_govt_data(mortgage_df, 0, 2, "%B %Y")
title_str = 'Mortgage Rates - ' + min(datetime_mortgage) + " to " + max(datetime_mortgage)

#create figure object
fig = px.line(mortgage_df, x=datetime_mortgage, y=values_mortgage)
#add interactive slider, title to X axis
fig.update_xaxes(rangeslider_visible=True, title_text="Date")
fig.update_yaxes(title_text='Mortgage Interest Rate')
fig.update_layout(title_text=title_str)

fig.show()

#### Interest Rate

In [21]:
filepath = "govt_data/interest-rate_jan-2019_may-2020.csv"
interest_df = pd.read_csv(filepath, skiprows=5)
interest_df.head()

,Unnamed: 0,Canada,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 498,Unnamed: 499,Unnamed: 500,Unnamed: 501,Unnamed: 502,Unnamed: 503,Unnamed: 504,Unnamed: 505,Unnamed: 506,Unnamed: 507
0,Financial market statistics,"January 1, 2019","January 2, 2019","January 3, 2019","January 4, 2019","January 5, 2019","January 6, 2019","January 7, 2019","January 8, 2019","January 9, 2019",...,"May 12, 2020","May 13, 2020","May 14, 2020","May 15, 2020","May 16, 2020","May 17, 2020","May 18, 2020","May 19, 2020","May 20, 2020","May 21, 2020"
1,NaN,Percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Overnight money market financing,..,1.7529,1.7518,1.7500,0.0000,0.0000,1.7507,1.7504,1.7479,...,0.2258,0.2259,0.2233,0.2233,0.0000,0.0000,..,0.2219,0.2295,..
3,"Government of Canada benchmark bond yields, 2 ...",..,1.86,1.77,1.84,0.00,0.00,1.88,1.89,1.91,...,0.27,0.28,0.27,0.28,0.00,0.00,..,0.30,0.30,0.30
4,"Government of Canada benchmark bond yields, 3 ...",..,1.86,1.77,1.84,0.00,0.00,1.88,1.89,1.90,...,0.26,0.26,0.25,0.26,0.00,0.00,..,0.29,0.29,0.29


In [22]:
datetime_int, values_int = parse_govt_data(interest_df, 0, 2, "%B %d, %Y")
title_str = 'Interest Rate - ' + min(datetime_int) + " to " + max(datetime_int)



#get rid of zero interest rate values on weekends and null values
values_int_adjusted = []
for i in range(len(values_int)):
    value = values_int[i]
    try:
        value_float = float(value)
        if value_float != 0.0:
            last_non_zero_value = value_float
            values_int_adjusted.append(value_float)
        else:
            values_int_adjusted.append(last_non_zero_value)
        
    except:
        values_int_adjusted.append(last_non_zero_value)
        
#print(values_int_adjusted)

assert len(values_int_adjusted) == len(values_int)
#     try:
#         float(value) == 0.0
#     except:
#         print(i)
#         print(values_int[-2])


In [23]:
#create figure object
fig = px.line(interest_df, x=datetime_int, y=values_int_adjusted)
#add interactive slider, title to X axis
fig.update_xaxes(rangeslider_visible=True, title_text="Date")
fig.update_yaxes(title_text='Overnight money market financing')
fig.update_layout(title_text=title_str)

fig.show()

#### Housing prices

In [24]:
#HOUSING PRICE IS RECORDED BY DIFF ORGANIZATION - STORED IN DIFFERENT FORMAT SO COLLECT MANUALLY
filepath = "govt_data/housing-prices_jan-2019_apr-2020.csv"
housing_df = pd.read_csv(filepath)
housing_df.head()
housing_df_relevant = housing_df.iloc[168:184]
housing_df_relevant

,Date,Composite_HPI,Single_Family_HPI,One_Storey_HPI,Two_Storey_HPI,Townhouse_HPI,Apartment_HPI,Composite_Benchmark,Single_Family_Benchmark,One_Storey_Benchmark,Two_Storey_Benchmark,Townhouse_Benchmark,Apartment_Benchmark
168,Jan 2019,227.4,225.9,220.2,229.6,227.3,231.5,584300.0,635600.0,504500.0,740000.0,481500.0,442200.0
169,Feb 2019,228.2,226.4,220.7,230.0,227.0,233.8,586300.0,637000.0,505700.0,741300.0,480800.0,446600.0
170,Mar 2019,229.8,228.4,222.9,231.9,228.3,234.4,590400.0,642700.0,510700.0,747400.0,483600.0,447800.0
171,Apr 2019,231.3,229.6,223.4,233.7,230.1,236.3,594300.0,646000.0,511800.0,753200.0,487400.0,451400.0
172,May 2019,232.0,230.6,225.1,234.2,231.5,236.1,596100.0,648900.0,515700.0,754800.0,490300.0,451000.0
173,Jun 2019,232.5,231.1,225.5,234.8,231.6,236.2,597400.0,650300.0,516700.0,756800.0,490600.0,451200.0
174,Jul 2019,232.6,231.2,226.0,234.7,232.0,236.3,597600.0,650600.0,517800.0,756400.0,491400.0,451400.0
175,Aug 2019,232.8,231.3,226.3,234.5,232.8,236.7,598100.0,650800.0,518500.0,755800.0,493100.0,452200.0
176,Sep 2019,233.3,231.9,226.5,235.5,233.2,237.0,599400.0,652500.0,518900.0,759000.0,493900.0,452800.0
177,Oct 2019,234.1,232.8,226.9,236.6,233.3,237.8,601500.0,655100.0,519900.0,762600.0,494200.0,454300.0


In [25]:
housing_datetimes = []
housing_values = []

for index, row in housing_df_relevant.iterrows():
    housing_dt = datetime.strptime(row["Date"], "%b %Y")
    housing_datetimes.append(housing_dt)
    housing_values.append(row["Composite_HPI"])

In [36]:
title_str = 'Housing prices - ' + str(min(housing_datetimes)) + " to " + str(max(housing_datetimes))

fig = px.line(x=housing_datetimes, y=housing_values)
#add interactive slider, title to X axis
fig.update_xaxes(rangeslider_visible=True, title_text="Date")
fig.update_yaxes(title_text='Overnight money market financing')
fig.update_layout(title_text=title_str)

fig.show()

#### Employment

In [29]:
filepath = "govt_data/employment_jan-2019_apr-2020.csv"
employment_df = pd.read_csv(filepath, skiprows=6)
employment_df.head()

#row of interest: 2 (employment of both sexes age 15 and over)

,Unnamed: 0,Unnamed: 1,Employment 3,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,Sex,Age group,January 2019,February 2019,March 2019,April 2019,May 2019,June 2019,July 2019,August 2019,September 2019,October 2019,November 2019,December 2019,January 2020,February 2020,March 2020,April 2020
1,NaN,NaN,Persons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Both sexes,15 years and over,"18,569.8","18,662.0","18,664.7","18,852.4","19,237.9","19,392.9","19,283.8","19,317.9","19,248.8","19,238.8","19,103.3","19,096.4","18,838.6","18,917.1","17,856.6","16,011.4"
3,NaN,15 to 24 years,"2,300.5","2,336.2","2,354.3","2,405.7","2,601.9","2,708.1","2,849.3","2,845.3","2,447.5","2,461.1","2,444.9","2,446.0","2,359.5","2,398.1","2,005.0","1,565.9"
4,NaN,25 years and over,"16,269.3","16,325.8","16,310.4","16,446.6","16,636.0","16,684.8","16,434.5","16,472.6","16,801.4","16,777.7","16,658.5","16,650.4","16,479.0","16,519.0","15,851.6","14,445.4"


In [32]:
datetime_emp, values_emp = parse_govt_data(employment_df, 0, 2, "%B %Y")
values_emp

['18,569.8',
 '18,662.0',
 '18,664.7',
 '18,852.4',
 '19,237.9',
 '19,392.9',
 '19,283.8',
 '19,317.9',
 '19,248.8',
 '19,238.8',
 '19,103.3',
 '19,096.4',
 '18,838.6',
 '18,917.1',
 '17,856.6',
 '16,011.4']

In [ ]:


fig = px.line(employment_df, x=datetime_emp, y=values_emp)
#add interactive slider, title to X axis
fig.update_xaxes(rangeslider_visible=True, title_text="Date")
fig.update_yaxes(title_text='Overnight money market financing')
fig.update_layout(title_text=title_str)

fig.show()

#### TSX

#### GDP